# Tweet Embeddings - OpenAI

## Libraries

In [18]:
# !pip install num2words
# !pip install azure-core
# !pip install azure-cosmos
# !pip install tiktoken
# !pip install openai

In [30]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
import openai
import configparser
from tqdm import tqdm
import ast

## Configuration File

In [20]:
config = configparser.ConfigParser()
config.read('config.ini')

openai.api_key = config['DEFAULT']['OPENAI_API_KEY']


## Data

In [21]:
tqdm.pandas()

In [22]:
data = pd.read_csv('../data/BBDD_SeAcabo.csv')
data

,Autora,full_text,Análisis General,Contenido Negativo,Insultos,Insulto 1,Insulto 2,Insulto 3,Emisor,Contenido AV.,...,fast_followers_count,normal_followers_count,followed_by,following,location,profile_banner_url,profile_url,avatar_url,verified,is_blue_verified
0,Jenni Hermoso,@Jennihermoso TODA ESPAÑA ESTÁ CONTIGO https:/...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,25903,NaN,NaN,NaN,https://pbs.twimg.com/profile_banners/13638585...,https://www.twitter.com/Araujismoo,https://pbs.twimg.com/profile_images/167886449...,False,True
1,Jenni Hermoso,@Jennihermoso Espero que te llegue todo el arr...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,194965,NaN,NaN,Navarra,https://pbs.twimg.com/profile_banners/34473932...,https://www.twitter.com/ionebelarra,https://pbs.twimg.com/profile_images/160626561...,False,False
2,Jenni Hermoso,@Jennihermoso No estás sola💜,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,91062,NaN,NaN,"Lleida, Catalunya, España.",https://pbs.twimg.com/profile_banners/23436984...,https://www.twitter.com/carlagaleote,https://pbs.twimg.com/profile_images/161793790...,False,False
3,Jenni Hermoso,@Araujismoo @Jennihermoso She’s a legend .,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3925,NaN,NaN,Earth,https://pbs.twimg.com/profile_banners/37527533...,https://www.twitter.com/emmaltrix,https://pbs.twimg.com/profile_images/169070741...,False,True
4,Jenni Hermoso,"@Jennihermoso Dilo, reina https://t.co/GasZHE70bE",Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,61457,NaN,NaN,"Las Palmas de Gran Canaria, Es",https://pbs.twimg.com/profile_banners/74429331...,https://www.twitter.com/TirodeGraciah,https://pbs.twimg.com/profile_images/163890386...,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,Eva Navarro,@evaaanavarro @Paulaa_311 @Jennihermoso 💜💪🏽,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,809,NaN,NaN,Torrelavega City,https://pbs.twimg.com/profile_banners/37359638...,https://www.twitter.com/sandrus260,https://pbs.twimg.com/profile_images/166507079...,False,False
2668,Eva Navarro,@evaaanavarro @Jennihermoso Z https://t.co/9vH...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,9820,NaN,NaN,Cataluña ✊💜💛♥️✊,https://pbs.twimg.com/profile_banners/14003913...,https://www.twitter.com/Giorgio7716,https://pbs.twimg.com/profile_images/164212369...,False,False
2669,Eva Navarro,@evaaanavarro @Jennihermoso Doblemente campeon...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2956,NaN,NaN,NaN,https://pbs.twimg.com/profile_banners/22259904...,https://www.twitter.com/framarub,https://pbs.twimg.com/profile_images/125216679...,False,False
2670,Eva Navarro,@evaaanavarro @Jennihermoso Brava!! Siempre co...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,182,NaN,NaN,NaN,https://pbs.twimg.com/profile_banners/90598249...,https://www.twitter.com/Joannacolomer,https://pbs.twimg.com/profile_images/140020219...,False,False


## Process Data

In [23]:
def process_data(text, sep_token = " \n "):
    # remove multiple spaces
    text = re.sub(r'\s+',  ' ', text).strip()
    text = re.sub(r"(.) ,",r"\1,",text)
    
    text = text.replace("..",".")
    text = text.replace(". .",".")
    text = text.replace("\n", "")
    text = text.strip()
    
    return text

In [24]:
data['full_text_processed'] = data["full_text"].progress_apply(lambda x : process_data(x))
data['full_text_processed']

0       @Jennihermoso TODA ESPAÑA ESTÁ CONTIGO https:/...
1       @Jennihermoso Espero que te llegue todo el arr...
2                            @Jennihermoso No estás sola💜
3              @Araujismoo @Jennihermoso She’s a legend .
4       @Jennihermoso Dilo, reina https://t.co/GasZHE70bE
                              ...                        
2667          @evaaanavarro @Paulaa_311 @Jennihermoso 💜💪🏽
2668    @evaaanavarro @Jennihermoso Z https://t.co/9vH...
2669    @evaaanavarro @Jennihermoso Doblemente campeon...
2670    @evaaanavarro @Jennihermoso Brava!! Siempre co...
2671    @evaaanavarro @Jennihermoso Y después de ver e...
Name: full_text_processed, Length: 2672, dtype: object

## Tokenize Data

In [25]:
tokenizer = tiktoken.get_encoding("cl100k_base")
data['n_tokens'] = data["full_text_processed"].progress_apply(lambda x: len(tokenizer.encode(x)))
len(data)

2672

In [26]:
data = data[data.n_tokens<8192]
len(data) # same as before

2672

## Create Embeddings

In [74]:
# Model name for embedding
model_name = "text-embedding-3-large"

def generate_embeddings(tweet, model=model_name):
    response = openai.Embedding.create(
        input=[tweet],
        model=model
    )
    return response['data'][0]['embedding']

In [28]:
data['embeddings'] = data["full_text_processed"].progress_apply(lambda x : generate_embeddings(x, model = model_name)) 
data

,Autora,full_text,Análisis General,Contenido Negativo,Insultos,Insulto 1,Insulto 2,Insulto 3,Emisor,Contenido AV.,...,following,location,profile_banner_url,profile_url,avatar_url,verified,is_blue_verified,full_text_processed,n_tokens,embeddings
0,Jenni Hermoso,@Jennihermoso TODA ESPAÑA ESTÁ CONTIGO https:/...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,https://pbs.twimg.com/profile_banners/13638585...,https://www.twitter.com/Araujismoo,https://pbs.twimg.com/profile_images/167886449...,False,True,@Jennihermoso TODA ESPAÑA ESTÁ CONTIGO https:/...,29,"[0.01613292284309864, 0.012918996624648571, -0..."
1,Jenni Hermoso,@Jennihermoso Espero que te llegue todo el arr...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Navarra,https://pbs.twimg.com/profile_banners/34473932...,https://www.twitter.com/ionebelarra,https://pbs.twimg.com/profile_images/160626561...,False,False,@Jennihermoso Espero que te llegue todo el arr...,35,"[0.006637951824814081, 0.005625439342111349, -..."
2,Jenni Hermoso,@Jennihermoso No estás sola💜,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Lleida, Catalunya, España.",https://pbs.twimg.com/profile_banners/23436984...,https://www.twitter.com/carlagaleote,https://pbs.twimg.com/profile_images/161793790...,False,False,@Jennihermoso No estás sola💜,13,"[0.008772128261625767, 0.0074376389384269714, ..."
3,Jenni Hermoso,@Araujismoo @Jennihermoso She’s a legend .,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Earth,https://pbs.twimg.com/profile_banners/37527533...,https://www.twitter.com/emmaltrix,https://pbs.twimg.com/profile_images/169070741...,False,True,@Araujismoo @Jennihermoso She’s a legend .,18,"[0.009303136728703976, 0.0092393197119236, 0.0..."
4,Jenni Hermoso,"@Jennihermoso Dilo, reina https://t.co/GasZHE70bE",Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Las Palmas de Gran Canaria, Es",https://pbs.twimg.com/profile_banners/74429331...,https://www.twitter.com/TirodeGraciah,https://pbs.twimg.com/profile_images/163890386...,False,True,"@Jennihermoso Dilo, reina https://t.co/GasZHE70bE",23,"[-0.024706777185201645, 0.02138485759496689, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,Eva Navarro,@evaaanavarro @Paulaa_311 @Jennihermoso 💜💪🏽,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Torrelavega City,https://pbs.twimg.com/profile_banners/37359638...,https://www.twitter.com/sandrus260,https://pbs.twimg.com/profile_images/166507079...,False,False,@evaaanavarro @Paulaa_311 @Jennihermoso 💜💪🏽,25,"[-0.015894215553998947, 0.020364463329315186, ..."
2668,Eva Navarro,@evaaanavarro @Jennihermoso Z https://t.co/9vH...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Cataluña ✊💜💛♥️✊,https://pbs.twimg.com/profile_banners/14003913...,https://www.twitter.com/Giorgio7716,https://pbs.twimg.com/profile_images/164212369...,False,False,@evaaanavarro @Jennihermoso Z https://t.co/9vH...,29,"[-0.02189185470342636, 0.028911961242556572, -..."
2669,Eva Navarro,@evaaanavarro @Jennihermoso Doblemente campeon...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,https://pbs.twimg.com/profile_banners/22259904...,https://www.twitter.com/framarub,https://pbs.twimg.com/profile_images/125216679...,False,False,@evaaanavarro @Jennihermoso Doblemente campeon...,27,"[0.017049280926585197, 0.016745861619710922, -..."
2670,Eva Navarro,@evaaanavarro @Jennihermoso Brava!! Siempre co...,Comentario Positivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,https://pbs.twimg.com/profile_banners/90598249...,https://www.twitter.com/Joannacolomer,https://pbs.twimg.com/profile_images/140020219...,False,False,@evaaanavarro @Jennihermoso Brava!! Siempre co...,36,"[0.01907341554760933, 0.018380094319581985, -0..."


In [29]:
# data.to_csv('seacabo_embeddings.csv', index = False, encoding = 'utf-8-sig')

## Visualize Embeddings

In [58]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.metrics import davies_bouldin_score, silhouette_score

In [36]:
# Convert embeddings to numpy array
embeddings = np.array(data['embeddings'].to_list())

### t-SNE

#### 2 Components

In [34]:
# Dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

In [ ]:
# # Plotting t-SNE results
# plt.figure(figsize=(10, 8))
# plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1])
# # for i, txt in enumerate(data["full_text_processed"]):
# #     plt.annotate(txt, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]))
# plt.title('t-SNE Visualization of Text Embeddings')
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.show()

In [44]:
# Agregar los componentes t-SNE al DataFrame
data['tsne-2d-one'] = embeddings_tsne[:, 0]
data['tsne-2d-two'] = embeddings_tsne[:, 1]

# Visualización interactiva con Plotly
fig = px.scatter(
    data,
    x='tsne-2d-one',
    y='tsne-2d-two',
    color='Análisis General',
    hover_data=['full_text_processed'],
    title='t-SNE Visualization of Text Embeddings'
)
fig.show()

#### 3 Components

In [54]:
tsne_3 = TSNE(n_components=3, random_state=42)
embeddings_tsne_3 = tsne_3.fit_transform(embeddings)

data['tsne-3d-one'] = embeddings_tsne_3[:, 0]
data['tsne-3d-two'] = embeddings_tsne_3[:, 1]
data['tsne-3d-three'] = embeddings_tsne_3[:, 2]

In [72]:
fig = px.scatter_3d(
    data,
    x='tsne-3d-one',
    y='tsne-3d-two',
    z='tsne-3d-three',
    color='Análisis General',
    hover_data=['full_text_processed'],
    title='3D t-SNE Visualization of Text Embeddings'
)
fig.update_traces(marker=dict(size=3))
fig.show()

### PCA

#### 2 Components

In [ ]:
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

In [ ]:
# plt.figure(figsize=(10, 8))
# plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1])
# # for i, txt in enumerate(data["full_text_processed"]):
# #     plt.annotate(txt, (embeddings_pca[i, 0], embeddings_pca[i, 1]))
# plt.title('PCA Visualization of Text Embeddings')
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.show()

In [50]:
# Agregar los componentes PCA al DataFrame
data['pca-2d-one'] = embeddings_pca[:, 0]
data['pca-2d-two'] = embeddings_pca[:, 1]

# Visualización interactiva con Plotly
fig = px.scatter(
    data,
    x='pca-2d-one',
    y='pca-2d-two',
    color='Análisis General',
    hover_data=['full_text_processed'],
    title='PCA Visualization of Text Embeddings'
)
fig.show()

#### 3 Components

In [52]:
pca_3 = PCA(n_components=3)
embeddings_pca_3 = pca_3.fit_transform(embeddings)
data['pca-3d-one'] = embeddings_pca_3[:, 0]
data['pca-3d-two'] = embeddings_pca_3[:, 1]
data['pca-3d-three'] = embeddings_pca_3[:, 2]

In [57]:
fig = px.scatter_3d(
    data,
    x='pca-3d-one',
    y='pca-3d-two',
    z='pca-3d-three',
    color='Análisis General',
    hover_data=['full_text_processed'],
    title='3D PCA Visualization of Text Embeddings'
)
fig.update_traces(marker=dict(size=3))
fig.show()

##### Metrics

In [61]:
# Calcular el índice Davies-Bouldin
db_index = davies_bouldin_score(embeddings_pca, data['Análisis General'])
print(f"Índice Davies-Bouldin: {db_index}")  # Cuanto más bajo mejor

Índice Davies-Bouldin: 2.7666636540821448


In [63]:
# Calcular el Coeficiente de Silueta
sil_score = silhouette_score(embeddings_pca, data['Análisis General'])
print(f"Coeficiente de Silueta: {sil_score}")  # Cuanto más cerca de uno mejor

Coeficiente de Silueta: 0.03397309493520737


# Word Embeddings - OpenAI

In [96]:
import time

def generate_word_embeddings(text, model=model_name, max_retries=5):
    for retry in range(max_retries):
        try:
            response = openai.Embedding.create(
                input=[text],
                model=model
            )
            return response['data'][0]['embedding']
        except openai.error.ServiceUnavailableError:
            wait_time = 2 ** retry  # Exponential backoff
            print(f"Service unavailable. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    raise Exception("Max retries exceeded")

In [97]:
# Extract unique words and generate embeddings for each word
unique_words = set()
for tweet in data['full_text_processed']:
    unique_words.update(tweet.split())

word_embeddings = {word: generate_word_embeddings(word) for word in unique_words}

# Convert word embeddings to a DataFrame
words = list(word_embeddings.keys())
word_embeddings_array = np.array([word_embeddings[word] for word in words])

# Create a DataFrame with word embeddings
df_word_embeddings = pd.DataFrame(word_embeddings_array, index=words)

# Export word embeddings to CSV
df_word_embeddings.to_csv('word_embeddings.csv')

Service unavailable. Retrying in 1 seconds...
Service unavailable. Retrying in 1 seconds...
Service unavailable. Retrying in 1 seconds...


KeyboardInterrupt: 

In [ ]:
# Reduce dimensionality using t-SNE for visualization
tsne_words = TSNE(n_components=2, random_state=42, perplexity=5)
embeddings_tsne_words = tsne_words.fit_transform(word_embeddings_array)

# Create a DataFrame for visualization
df_words_tsne = pd.DataFrame({
    'word': words,
    'x': embeddings_tsne_words[:, 0],
    'y': embeddings_tsne_words[:, 1]
})

# 2D Visualization with Plotly for words
fig_words = px.scatter(
    df_words_tsne,
    x='x',
    y='y',
    text='word',
    title='t-SNE Visualization of Word Embeddings'
)
fig_words.update_traces(marker=dict(size=10))
fig_words.show()

# Compare Words

In [88]:
# Generate embeddings for the two words
word1 = "jefe"
word2 = "rubi"

embedding_word1 = generate_embeddings(word1)
embedding_word2 = generate_embeddings(word2)

# Convert embeddings to numpy arrays
embedding_word1 = np.array(embedding_word1).reshape(1, -1)
embedding_word2 = np.array(embedding_word2).reshape(1, -1)

In [89]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
similarity = cosine_similarity(embedding_word1, embedding_word2)[0][0]
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity}")


Cosine similarity between 'jefe' and 'rubi': 0.2839960394803581


In [90]:
# Combine embeddings and labels for t-SNE
embeddings_words = np.vstack([embedding_word1, embedding_word2])
labels = [word1, word2]

# Reduce dimensionality using t-SNE
tsne_words = TSNE(n_components=2, random_state=42, perplexity=1)
embeddings_tsne_words = tsne_words.fit_transform(embeddings_words)

# Create a DataFrame for visualization
df = pd.DataFrame({
    'word': labels,
    'x': embeddings_tsne_words[:, 0],
    'y': embeddings_tsne_words[:, 1]
})

# 2D Visualization with Plotly
fig = px.scatter(
    df,
    x='x',
    y='y',
    text='word',
    title='t-SNE Visualization of Word Embeddings'
)
fig.update_traces(marker=dict(size=10))
fig.show()